In [1]:
# prompt: mount drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, Normalizer
from sklearn.svm import SVC
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import torch.optim as optim
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from datetime import datetime
import os
import joblib

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

import torchvision.transforms as T
from torchvision.models import resnet18, ResNet18_Weights

import torchvision.models as models
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights

In [3]:
# 1. Load CSV
df = pd.read_csv('/content/drive/MyDrive/AML-PROJECT/623final_all.csv')


# 2. Extract ID
df['ID'] = df['label'].str.extract(r'(\d+)', expand=False)

# Optional: Save filtered train/val
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['ID'], random_state=42)

# 3. Encode labels
le = LabelEncoder()
train_df['encoded_label'] = le.fit_transform(train_df['label'])
val_df['encoded_label'] = le.transform(val_df['label'])

### Resnet18 on Noise Dataset

In [4]:
# 4. Dataset class
class IrisDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.iloc[idx]['image_path']
        label = self.df.iloc[idx]['encoded_label']
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

# 5.Define transformations and loaders
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

train_dataset = IrisDataset(train_df, transform=transform)
val_dataset = IrisDataset(val_df, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


# 6. Load ResNet18 and modify final layer
model = models.resnet18(pretrained=True)
num_classes = len(le.classes_)
model.fc = nn.Linear(model.fc.in_features, num_classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


# 7.Loss function
# Define Focal Loss
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.ce = nn.CrossEntropyLoss(reduction='none')

    def forward(self, input, target):
        ce_loss = self.ce(input, target)
        pt = torch.exp(-ce_loss)
        focal_loss = ((1 - pt) ** self.gamma) * ce_loss
        return focal_loss.mean()

# Define Label Smoothing Loss
class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=-1)
        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=-1))

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 193MB/s]


In [5]:
# 8. Evaluation Function
def evaluate(model, loader, device):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            outputs = model(x)
            preds = outputs.argmax(dim=1)
            y_true.extend(y.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'f1': f1_score(y_true, y_pred, average='weighted',zero_division=0),
        'precision': precision_score(y_true, y_pred, average='weighted',zero_division=0),
        'recall': recall_score(y_true, y_pred, average='weighted',zero_division=0)
    }

# 9. Training Function
def train_resnet18(loss_fn, optimizer_cls, epochs=5):
    model = models.resnet18(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    model = model.to(device)

    optimizer = optimizer_cls(model.parameters(), lr=1e-3)

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for x, y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            outputs = model(x)
            loss = loss_fn(outputs, y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        metrics = evaluate(model, val_loader, device)
        print(f"Epoch {epoch+1} | Loss: {total_loss/len(train_loader):.4f} | "
              f"Acc: {metrics['accuracy']:.4f} | F1: {metrics['f1']:.4f}")

    final_metrics = evaluate(model, val_loader, device)


    timestamp = datetime.now().strftime('%Y%m%d-%H%M%S')
    model_name = f"resnet18__{loss_fn.__class__.__name__}__{optimizer_cls.__name__}__{timestamp}.pth"

    save_dir = "/content/drive/MyDrive/AML-PROJECT/pretrained_cnn_Ly/model2"
    os.makedirs(save_dir, exist_ok=True)
    model_path = os.path.join(save_dir, model_name)

    torch.save(model.state_dict(), model_path)
    print(f"Saved model to: {model_path}")


    return {
        'loss_fn': loss_fn.__class__.__name__,
        'optimizer': optimizer_cls.__name__ if hasattr(optimizer_cls, '__name__') else str(optimizer_cls),
        'accuracy': final_metrics['accuracy'],
        'f1': final_metrics['f1'],
        'precision': final_metrics['precision'],
        'recall': final_metrics['recall'],
        'timestamp': datetime.now().isoformat()
    }

In [6]:
# 10. Run Experiments
loss_fns = [
    nn.CrossEntropyLoss(),
    LabelSmoothingLoss(classes=num_classes),
    FocalLoss(gamma=2)
]

optimizers = [optim.Adam, optim.SGD]

results = []

for loss_fn in loss_fns:
    for opt in optimizers:
        try:
            print(f"\nRunning: {loss_fn.__class__.__name__} + {opt.__name__}")
            result = train_resnet18(loss_fn, opt, epochs=5)
            results.append(result)
        except Exception as e:
            print(f"[ERROR] Skipped: {loss_fn}, {opt} — {e}")


Running: CrossEntropyLoss + Adam


Epoch 1/5: 100%|██████████| 50/50 [19:25<00:00, 23.30s/it]


Epoch 1 | Loss: 5.2454 | Acc: 0.0575 | F1: 0.0347


Epoch 2/5: 100%|██████████| 50/50 [00:07<00:00,  6.96it/s]


Epoch 2 | Loss: 3.1189 | Acc: 0.2975 | F1: 0.2485


...

Epoch 3 | Loss: 5.2525 | Acc: 0.0075 | F1: 0.0016


Epoch 4/5: 100%|██████████| 50/50 [00:07<00:00,  7.03it/s]


Epoch 4 | Loss: 5.1721 | Acc: 0.0075 | F1: 0.0012


Epoch 5/5: 100%|██████████| 50/50 [00:07<00:00,  7.13it/s]


Epoch 5 | Loss: 5.0912 | Acc: 0.0100 | F1: 0.0034
Saved model to: /content/drive/MyDrive/AML-PROJECT/pretrained_cnn_Ly/model2/resnet18__FocalLoss__SGD__20250705-144710.pth


In [7]:
#  11. Summary Table
df_results = pd.DataFrame(results)
df_results = df_results.sort_values(by="accuracy", ascending=False)

print("\n=== SUMMARY RESULTS ===")
print(df_results)


=== SUMMARY RESULTS ===
              loss_fn optimizer  accuracy        f1  precision  recall  \
2  LabelSmoothingLoss      Adam    0.8425  0.833089   0.868583  0.8425   
0    CrossEntropyLoss      Adam    0.8375  0.824595   0.855629  0.8375   
4           FocalLoss      Adam    0.7200  0.703169   0.773347  0.7200   
1    CrossEntropyLoss       SGD    0.0100  0.012143   0.019375  0.0100   
5           FocalLoss       SGD    0.0100  0.003413   0.002463  0.0100   
3  LabelSmoothingLoss       SGD    0.0050  0.002833   0.002167  0.0050   

                    timestamp  
2  2025-07-05T14:44:56.237508  
0  2025-07-05T14:43:26.416919  
4  2025-07-05T14:46:25.978279  
1  2025-07-05T14:44:11.028072  
5  2025-07-05T14:47:10.877369  
3  2025-07-05T14:45:40.748115  


## Resnet 18_1 Feature Extraction + svm/logistic on Noise Dataset

In [16]:
save_dir = "/content/drive/MyDrive/AML-PROJECT/pretrained_cnn_Ly/Resnet18_1_Noise"
os.makedirs(save_dir, exist_ok=True)

# 1. Data Loading

class IrisDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.iloc[idx]['image_path']
        label = self.df.iloc[idx]['encoded_label']
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

# Data transformations (matching ResNet input requirements)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


train_dataset = IrisDataset(train_df, transform=transform)
val_dataset = IrisDataset(val_df, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False) # shuffle=False to maintain feature order
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [9]:
# 2.  CNN Feature Extraction (using pretrained ResNet18 without fine-tuning)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define ResNet18 feature extractor
class ResNet18FeatureExtractor(nn.Module):
    def __init__(self):
        super().__init__()
        self.resnet = models.resnet18(pretrained=True)
        self.features = nn.Sequential(*list(self.resnet.children())[:-1]) # Remove final layer

    def forward(self, x):
        x = self.features(x)
        return torch.flatten(x, 1)  # Flatten into feature vector

# Global model initialization
model = ResNet18FeatureExtractor().to(device).eval()

def extract_features(dataloader, model): # Accept model as parameter
    features, labels = [], []
    with torch.no_grad():
        for images, lbls in tqdm(dataloader,desc="Extracting Features"):
            images = images.to(device)
            feats = model(images).cpu().numpy()
            features.append(feats)
            labels.append(lbls.numpy())
    return np.vstack(features), np.concatenate(labels)

print("Extracting train features...")
train_features, train_labels = extract_features(train_loader, model)
print("Extracting test features...")
val_features, val_labels = extract_features(val_loader, model)

# Save extracted features
np.save(os.path.join(save_dir, 'train_features.npy'), train_features)
np.save(os.path.join(save_dir, 'train_labels.npy'), train_labels)
np.save(os.path.join(save_dir, 'val_features.npy'), val_features)
np.save(os.path.join(save_dir, 'val_labels.npy'), val_labels)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Extracting train features...


Extracting Features: 100%|██████████| 50/50 [00:06<00:00,  7.55it/s]


Extracting test features...


Extracting Features: 100%|██████████| 13/13 [00:01<00:00,  8.14it/s]


In [10]:
# 3. Train Classifiers (SVM and Logistic Regression)
# Feature standardization
scaler = StandardScaler()
train_features_scaled = scaler.fit_transform(train_features)
val_features_scaled = scaler.transform(val_features)

# Save the scaler
joblib.dump(scaler, os.path.join(save_dir, 'feature_scaler.pkl'))

# Initialize classifiers
svm = SVC(C=10, kernel='rbf', gamma='scale', probability=True)
lr = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)

print("\nTraining classifiers...")
svm.fit(train_features_scaled, train_labels)
lr.fit(train_features_scaled, train_labels)

# Save trained models
joblib.dump(svm, os.path.join(save_dir, 'svm_classifier.pkl'))
joblib.dump(lr, os.path.join(save_dir, 'logistic_regression_classifier.pkl'))


Training classifiers...


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


['/content/drive/MyDrive/AML-PROJECT/pretrained_cnn_Ly/Resnet18_1_Noise/logistic_regression_classifier.pkl']

In [11]:
# 4. Evaluation Result

def evaluate_model(model, X_val, y_val, model_name):
    y_pred = model.predict(X_val)
    return {
        'variant': model_name,
        'accuracy': accuracy_score(y_val, y_pred),
        'f1': f1_score(y_val, y_pred, average='weighted', zero_division=0),
        'precision': precision_score(y_val, y_pred, average='weighted', zero_division=0),
        'recall': recall_score(y_val, y_pred, average='weighted', zero_division=0)
    }

results = [
    evaluate_model(lr, val_features_scaled, val_labels, "Logistic"),
    evaluate_model(svm, val_features_scaled, val_labels, "SVM_rbf")
]

# Format output
df_results = pd.DataFrame(results)
print("\n=== SUMMARY RESULTS (ResNet18 Features + ML Classifiers) ===")
print(df_results.to_string(
    index=False,
    columns=['variant', 'accuracy', 'f1', 'precision', 'recall'],
    float_format="%.4f"
))

torch.save(model.state_dict(), os.path.join(save_dir, 'resnet18_feature_extractor.pth'))
print(f"\nAll models and results saved to: {save_dir}")


=== SUMMARY RESULTS (ResNet18 Features + ML Classifiers) ===
 variant  accuracy     f1  precision  recall
Logistic    0.4650 0.4413     0.4749  0.4650
 SVM_rbf    0.4050 0.3808     0.4316  0.4050

All models and results saved to: /content/drive/MyDrive/AML-PROJECT/pretrained_cnn_Ly/Resnet18_1_Noise


## Resent18_2- Fine Tuning + Feature Extraction + SVM/LogReg on Noise Dataset

In [12]:
save_dir = "/content/drive/MyDrive/AML-PROJECT/pretrained_cnn_Ly/Resnet18_2_Noise"
os.makedirs(save_dir, exist_ok=True)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(">>> Device:", device)

# 1. Data Loading
CSV_PATH = '/content/drive/MyDrive/AML-PROJECT/623final_all.csv'
BATCH_SIZE = 32
NUM_WORKERS = 2

df = pd.read_csv(CSV_PATH)
df['ID'] = df['label'].str.extract(r'(\d+)', expand=False)
train_df, val_df = train_test_split(
    df, test_size=0.2, stratify=df['ID'], random_state=42)

le = LabelEncoder()
train_df['encoded_label'] = le.fit_transform(train_df['label'])
val_df['encoded_label']   = le.transform(val_df['label'])
num_classes = len(le.classes_)
print(f">>> Num classes: {num_classes}")

class IrisDataset(Dataset):
    def __init__(self, df, tfm):
        self.df, self.tfm = df.reset_index(drop=True), tfm
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        img = Image.open(self.df.loc[idx,'image_path']).convert('RGB')
        return self.tfm(img), self.df.loc[idx,'encoded_label']

tfm = T.Compose([
    T.Resize((224,224)),
    T.ToTensor(),
    T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

train_loader = DataLoader(IrisDataset(train_df,tfm), batch_size=BATCH_SIZE,
                          shuffle=True, num_workers=NUM_WORKERS)
val_loader   = DataLoader(IrisDataset(val_df,tfm), batch_size=BATCH_SIZE,
                          shuffle=False, num_workers=NUM_WORKERS)


# 2. Custom Losses
class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.1):
        super().__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing  = smoothing
        self.classes    = classes
    def forward(self, pred, target):
        pred = pred.log_softmax(dim=-1)
        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.classes-1))
            true_dist.scatter_(1, target.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist*pred, dim=-1))

class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0):
        super().__init__()
        self.gamma = gamma
        self.ce = nn.CrossEntropyLoss(reduction='none')
    def forward(self, pred, target):
        ce = self.ce(pred, target)
        pt = torch.exp(-ce)
        return ((1-pt)**self.gamma * ce).mean()

# 3. Loss/Optimizer Combinations

loss_fns = [
    ('CE', nn.CrossEntropyLoss()),
    ('LS', LabelSmoothingLoss(num_classes)),
    ('FL', FocalLoss(gamma=2))
]
opt_fns = [
    ('Adam', lambda p: optim.Adam(p, lr=3e-4, weight_decay=1e-4)),
    ('SGD' , lambda p: optim.SGD(p, lr=0.01, momentum=0.9, weight_decay=1e-4))
]
EPOCHS = 5


# 4. Encoder for ResNet18

def make_encoder(model):
    return nn.Sequential(
        model.conv1, model.bn1, model.relu, model.maxpool,
        model.layer1, model.layer2, model.layer3, model.layer4,
        model.avgpool, nn.Flatten()
    ).eval().to(device)

def extract(loader, enc):
    feats, lbls = [], []
    with torch.no_grad():
        for x,y in loader:
            feats.append(enc(x.to(device)).cpu().numpy())
            lbls.append(y.numpy())
    return np.vstack(feats), np.concatenate(lbls)

# 5. Training + Feature Extraction + Evaluation + Model Saving
results = []
for lname, loss_fn in loss_fns:
    for oname, opt_fn in opt_fns:
        print(f"\n=== {lname} + {oname} ===")

        # Create subdirectory for current configuration
        config_dir = os.path.join(save_dir, f"{lname}_{oname}")
        os.makedirs(config_dir, exist_ok=True)

        # Model initialization
        model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        model.fc = nn.Linear(model.fc.in_features, num_classes)
        for n,p in model.named_parameters():
            p.requires_grad = n.startswith('layer4') or n.startswith('fc')
        model = model.to(device)
        optimizer = opt_fn(filter(lambda p: p.requires_grad, model.parameters()))

        # Training loop
        for ep in range(1, EPOCHS+1):
            model.train(); tot, corr, total = 0,0,0
            for x,y in train_loader:
                x,y = x.to(device), y.to(device)
                optimizer.zero_grad()
                out = model(x)
                loss = loss_fn(out, y)
                loss.backward(); optimizer.step()
                tot += loss.item()
                corr += (out.argmax(1)==y).sum().item(); total += y.size(0)
            print(f"  Epoch {ep}/{EPOCHS} | TrainAcc {(corr/total):.4f}")

        # Save complete model and trained weights
        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss_fn.state_dict() if hasattr(loss_fn, 'state_dict') else None,
            'label_encoder': le.classes_  # Save label encoding info
        }, os.path.join(config_dir, 'full_model.pth'))

        # Feature extraction and normalization
        enc = make_encoder(model)
        X_tr,y_tr = extract(train_loader, enc)
        X_va,y_va = extract(val_loader,   enc)
        norm = Normalizer('l2')
        X_tr = norm.fit_transform(X_tr); X_va = norm.transform(X_va)

        # Save feature extractor and normalizer
        torch.save(enc.state_dict(), os.path.join(config_dir, 'feature_extractor.pth'))
        joblib.dump(norm, os.path.join(config_dir, 'normalizer.joblib'))

        # Train and save classifiers
        svc = SVC(C=10, kernel='rbf', gamma='scale', probability=True).fit(X_tr,y_tr)
        logreg = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000).fit(X_tr,y_tr)

        joblib.dump(svc, os.path.join(config_dir, 'svc_classifier.joblib'))
        joblib.dump(logreg, os.path.join(config_dir, 'logreg_classifier.joblib'))

        # Evaluation
        def evaluate(model, name):
            pred = model.predict(X_va)
            return {
                'Model'    : f"ResNet18 + {name}",
                'Loss'     : lname,
                'Opt'      : oname,
                'Accuracy' : accuracy_score(y_va, pred),
                'F1'       : f1_score(y_va, pred, average='weighted', zero_division=0),
                'Precision': precision_score(y_va, pred, average='weighted', zero_division=0),
                'Recall'   : recall_score(y_va, pred, average='weighted', zero_division=0)
            }

        results.append(evaluate(svc, "SVC"))
        results.append(evaluate(logreg, "LogReg"))

        print(f"Model and components saved to: {config_dir}")

# 6. Save final results and configuration
df_results = pd.DataFrame(results)
df_results.to_csv(os.path.join(save_dir, 'all_results.csv'), index=False)

# Save data preprocessing info
preprocess_info = {
    'image_size': 224,
    'normalization_mean': [0.485, 0.456, 0.406],
    'normalization_std': [0.229, 0.224, 0.225],
    'label_encoder': le.classes_
}
joblib.dump(preprocess_info, os.path.join(save_dir, 'preprocess_info.joblib'))

print("\n=== SUMMARY RESULTS ===")
print(df_results.sort_values('Accuracy', ascending=False).to_string(index=False, float_format="%.4f"))
print(f"\nAll models and results saved to: {save_dir}")

>>> Device: cuda
>>> Num classes: 200

=== CE + Adam ===
  Epoch 1/5 | TrainAcc 0.0919
  Epoch 2/5 | TrainAcc 0.8356
  Epoch 3/5 | TrainAcc 0.9950
  Epoch 4/5 | TrainAcc 1.0000
  Epoch 5/5 | TrainAcc 1.0000


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Model and components saved to: /content/drive/MyDrive/AML-PROJECT/pretrained_cnn_Ly/Resnet18_2_Noise/CE_Adam

=== CE + SGD ===
  Epoch 1/5 | TrainAcc 0.0319
  Epoch 2/5 | TrainAcc 0.5031
  Epoch 3/5 | TrainAcc 0.9406
  Epoch 4/5 | TrainAcc 0.9994
  Epoch 5/5 | TrainAcc 1.0000


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Model and components saved to: /content/drive/MyDrive/AML-PROJECT/pretrained_cnn_Ly/Resnet18_2_Noise/CE_SGD

=== LS + Adam ===
  Epoch 1/5 | TrainAcc 0.0956
  Epoch 2/5 | TrainAcc 0.8688
  Epoch 3/5 | TrainAcc 0.9981
  Epoch 4/5 | TrainAcc 1.0000
  Epoch 5/5 | TrainAcc 1.0000


...

Model and components saved to: /content/drive/MyDrive/AML-PROJECT/pretrained_cnn_Ly/Resnet18_2_Noise/LS_SGD

=== FL + Adam ===
  Epoch 1/5 | TrainAcc 0.0887
  Epoch 2/5 | TrainAcc 0.8144
  Epoch 3/5 | TrainAcc 0.9931
  Epoch 4/5 | TrainAcc 1.0000
  Epoch 5/5 | TrainAcc 1.0000


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Model and components saved to: /content/drive/MyDrive/AML-PROJECT/pretrained_cnn_Ly/Resnet18_2_Noise/FL_Adam

=== FL + SGD ===
  Epoch 1/5 | TrainAcc 0.0306
  Epoch 2/5 | TrainAcc 0.5106
  Epoch 3/5 | TrainAcc 0.9375
  Epoch 4/5 | TrainAcc 0.9988
  Epoch 5/5 | TrainAcc 1.0000


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Model and components saved to: /content/drive/MyDrive/AML-PROJECT/pretrained_cnn_Ly/Resnet18_2_Noise/FL_SGD

=== SUMMARY RESULTS ===
            Model Loss  Opt  Accuracy     F1  Precision  Recall
   ResNet18 + SVC   CE Adam    0.7350 0.7214     0.7729  0.7350
   ResNet18 + SVC   FL Adam    0.7325 0.7293     0.8098  0.7325
   ResNet18 + SVC   LS Adam    0.7200 0.7102     0.7637  0.7200
   ResNet18 + SVC   CE  SGD    0.6875 0.6734     0.7219  0.6875
   ResNet18 + SVC   FL  SGD    0.6875 0.6703     0.7249  0.6875
   ResNet18 + SVC   LS  SGD    0.6825 0.6740     0.7454  0.6825
ResNet18 + LogReg   CE Adam    0.6475 0.6433     0.7429  0.6475
ResNet18 + LogReg   FL Adam    0.5975 0.5854     0.6881  0.5975
ResNet18 + LogReg   LS Adam    0.5900 0.5818     0.6760  0.5900
ResNet18 + LogReg   LS  SGD    0.5200 0.5133     0.6268  0.5200
ResNet18 + LogReg   CE  SGD    0.5075 0.4939     0.5860  0.5075
ResNet18 + LogReg   FL  SGD    0.4925 0.4726     0.5630  0.4925

All models and results saved to: /

##  EfficientNet_1 Feature Extraction +SVM/ LogReg on Noise Dataset

In [14]:
# === Set up save directory ===
save_dir = "/content/drive/MyDrive/AML-PROJECT/pretrained_cnn_Ly/EfficientNet_1_Noise"
os.makedirs(save_dir, exist_ok=True)
print(f">>> Models will be saved to: {save_dir}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(">>> Using device:", device)

# === Paths and parameters ===
CSV_PATH     = '/content/drive/MyDrive/AML-PROJECT/623final_all.csv'
BATCH_SIZE   = 32
NUM_WORKERS  = 2   # Change to 0 on Windows

# === Step 1: Data loading & label processing ===
df = pd.read_csv(CSV_PATH)
df['ID'] = df['label'].str.extract(r'(\d+)', expand=False)

train_df, val_df = train_test_split(
    df, test_size=0.2, stratify=df['ID'], random_state=42
)

le = LabelEncoder()
train_df['encoded_label'] = le.fit_transform(train_df['label'])
val_df['encoded_label']   = le.transform(val_df['label'])
print(f">>> Num classes: {len(le.classes_)}")

# Save label encoder
joblib.dump(le, os.path.join(save_dir, 'label_encoder.joblib'))

# === Step 2: Dataset & DataLoader ===
class IrisDataset(Dataset):
    def __init__(self, df, tfm):
        self.df = df.reset_index(drop=True)
        self.tfm = tfm

    def __len__(self): return len(self.df)

    def __getitem__(self, idx):
        img = Image.open(self.df.loc[idx, 'image_path']).convert('RGB')
        img = self.tfm(img)
        label = self.df.loc[idx, 'encoded_label']
        return img, label

transform = T.Compose([
    T.Resize((224, 224)),  # EfficientNet B0 default input size
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406],
                [0.229, 0.224, 0.225])
])

# Save transform info
joblib.dump({
    'image_size': 224,
    'normalization_mean': [0.485, 0.456, 0.406],
    'normalization_std': [0.229, 0.224, 0.225]
}, os.path.join(save_dir, 'transform_info.joblib'))

train_loader = DataLoader(IrisDataset(train_df, transform),
                          batch_size=BATCH_SIZE, shuffle=False,
                          num_workers=NUM_WORKERS)
val_loader   = DataLoader(IrisDataset(val_df,   transform),
                          batch_size=BATCH_SIZE, shuffle=False,
                          num_workers=NUM_WORKERS)

# === Step 3: EfficientNet-B0 Feature Extractor ===
class EffNetB0Extractor(nn.Module):
    def __init__(self):
        super().__init__()
        model = efficientnet_b0(weights=EfficientNet_B0_Weights.IMAGENET1K_V1)
        self.features = nn.Sequential(model.features, model.avgpool)  # Output (B, 1280, 1, 1)
        for p in self.features.parameters():
            p.requires_grad = False
        self.features.eval()

    def forward(self, x):
        with torch.no_grad():
            x = self.features(x)
            x = torch.flatten(x, 1)
        return x

feature_extractor = EffNetB0Extractor().to(device)

# Save feature extractor
torch.save(feature_extractor.state_dict(), os.path.join(save_dir, 'efficientnet_b0_extractor.pth'))

def extract_features(loader):
    feats, labels = [], []
    for imgs, lbls in tqdm(loader, desc="► Extracting features"):
        imgs = imgs.to(device)
        emb  = feature_extractor(imgs).cpu().numpy()
        feats.append(emb)
        labels.append(lbls.numpy())
    return np.vstack(feats), np.concatenate(labels)

print("\n[Stage] Feature extraction...")
X_train_raw, y_train = extract_features(train_loader)
X_val_raw,   y_val   = extract_features(val_loader)

# Save extracted features
np.save(os.path.join(save_dir, 'train_features.npy'), X_train_raw)
np.save(os.path.join(save_dir, 'val_features.npy'), X_val_raw)
np.save(os.path.join(save_dir, 'train_labels.npy'), y_train)
np.save(os.path.join(save_dir, 'val_labels.npy'), y_val)

# === Step 4: L2 Normalization + Train Classifiers ===
l2 = Normalizer('l2')
X_train = l2.fit_transform(X_train_raw)
X_val   = l2.transform(X_val_raw)

# Save normalizer
joblib.dump(l2, os.path.join(save_dir, 'l2_normalizer.joblib'))

# Train and save classifiers
svc_rbf = SVC(kernel='rbf', C=10, gamma='scale')
svc_rbf.fit(X_train, y_train)
joblib.dump(svc_rbf, os.path.join(save_dir, 'rbf_svc.joblib'))

logreg = LogisticRegression(multi_class='multinomial', solver='lbfgs',
                            max_iter=1000, C=10)
logreg.fit(X_train, y_train)
joblib.dump(logreg, os.path.join(save_dir, 'logistic_regression.joblib'))

# === Step 5: Evaluation and Reporting ===
def evaluate(model, name):
    pred = model.predict(X_val)
    return {
        'Model'    : name,
        'Accuracy' : accuracy_score(y_val, pred),
        'F1'       : f1_score(y_val, pred, average='weighted', zero_division=0),
        'Precision': precision_score(y_val, pred, average='weighted', zero_division=0),
        'Recall'   : recall_score(y_val, pred, average='weighted', zero_division=0)
    }

results = [
    evaluate(svc_rbf, "EffB0 + RBF SVM"),
    evaluate(logreg, "EffB0 + Logistic")
]

df_results = pd.DataFrame(results)
print("\n=== SUMMARY RESULTS ===")
print(df_results.to_string(index=False, float_format="%.4f"))

# Save results
df_results.to_csv(os.path.join(save_dir, 'evaluation_results.csv'), index=False)

print(f"\n>>> All models and results saved to: {save_dir}")

>>> Models will be saved to: /content/drive/MyDrive/AML-PROJECT/pretrained_cnn_Ly/EfficientNet_1_Noise
>>> Using device: cuda
>>> Num classes: 200

[Stage] Feature extraction...


► Extracting features: 100%|██████████| 13/13 [00:01<00:00, 11.92it/s]
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(



=== SUMMARY RESULTS ===
           Model  Accuracy     F1  Precision  Recall
 EffB0 + RBF SVM    0.4750 0.4623     0.5292  0.4750
EffB0 + Logistic    0.4325 0.4157     0.4970  0.4325

>>> All models and results saved to: /content/drive/MyDrive/AML-PROJECT/pretrained_cnn_Ly/EfficientNet_1_Noise


## EfficientNet_2 Feature Extraction +  Fine Tuning + SVM/LogReg on Noise Dataset

In [17]:
import datetime

save_dir = "/content/drive/MyDrive/AML-PROJECT/pretrained_cnn_Ly/EfficientNet_2_Noise"
os.makedirs(save_dir, exist_ok=True)
print(f">>> All models will be saved to: {save_dir}")


# EfficientNet-B0 Fine-tune + Feature Extraction + Linear Classifier Evaluation

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(">>> Device:", device)

# 1. Load CSV and process labels

CSV_PATH = '/content/drive/MyDrive/AML-PROJECT/623final_all.csv'
BATCH_SIZE = 32
NUM_WORKERS = 2

# Load and process DataFrame
df = pd.read_csv(CSV_PATH)
df['ID'] = df['label'].str.extract(r'(\d+)', expand=False)
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['ID'], random_state=42)

# Encode labels
le = LabelEncoder()
train_df['encoded_label'] = le.fit_transform(train_df['label'])
val_df['encoded_label'] = le.transform(val_df['label'])
num_classes = len(le.classes_)
print(f">>> Num classes: {num_classes}")

# Save label encoder
joblib.dump(le, os.path.join(save_dir, 'label_encoder.joblib'))

# Define dataset
class IrisDataset(Dataset):
    def __init__(self, df, transform):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img = Image.open(self.df.loc[idx, 'image_path']).convert('RGB')
        return self.transform(img), self.df.loc[idx, 'encoded_label']

# Define image transform
transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Save transform info
joblib.dump({
    'image_size': 224,
    'normalization_mean': [0.485, 0.456, 0.406],
    'normalization_std': [0.229, 0.224, 0.225]
}, os.path.join(save_dir, 'transform_info.joblib'))

# Loaders
train_loader = DataLoader(IrisDataset(train_df, transform), batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
val_loader = DataLoader(IrisDataset(val_df, transform), batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)


# 2. Custom Loss Functions

class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.1):
        super().__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.classes = classes

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=-1)
        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.classes - 1))
            true_dist.scatter_(1, target.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=-1))

class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0):
        super().__init__()
        self.gamma = gamma
        self.ce = nn.CrossEntropyLoss(reduction='none')

    def forward(self, pred, target):
        ce = self.ce(pred, target)
        pt = torch.exp(-ce)
        return ((1 - pt) ** self.gamma * ce).mean()

# Loss and optimizer combinations
loss_fns = [
    ('CE', nn.CrossEntropyLoss()),
    ('LS', LabelSmoothingLoss(num_classes)),
    ('FL', FocalLoss(gamma=2))
]
opt_fns = [
    ('Adam', lambda p: optim.Adam(p, lr=3e-4, weight_decay=1e-4)),
    ('SGD', lambda p: optim.SGD(p, lr=0.01, momentum=0.9, weight_decay=1e-4))
]

EPOCHS = 10


# 3. Feature Extractor & Evaluator & Saving

def make_encoder(model):
    return nn.Sequential(model.features, model.avgpool, nn.Flatten()).eval().to(device)

def extract_features(loader, encoder):
    feats, labels = [], []
    with torch.no_grad():
        for x, y in loader:
            feats.append(encoder(x.to(device)).cpu().numpy())
            labels.append(y.numpy())
    return np.vstack(feats), np.concatenate(labels)

def evaluate(model, name, X, y):
    pred = model.predict(X)
    return {
        'Variant': name,
        'Accuracy': accuracy_score(y, pred),
        'F1': f1_score(y, pred, average='weighted', zero_division=0),
        'Precision': precision_score(y, pred, average='weighted', zero_division=0),
        'Recall': recall_score(y, pred, average='weighted', zero_division=0)
    }


# 4. Main Experiment Loop with Comprehensive Saving

results = []
for lname, loss_fn in loss_fns:
    for oname, opt_fn in opt_fns:
        config_name = f"{lname}_{oname}"
        config_dir = os.path.join(save_dir, config_name)
        os.makedirs(config_dir, exist_ok=True)
        print(f"\n=== {config_name} ===")

        # Model initialization
        model = efficientnet_b0(weights=EfficientNet_B0_Weights.IMAGENET1K_V1)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)

        # Freeze layers except last block and classifier
        for name, param in model.named_parameters():
            param.requires_grad = name.startswith('features.6') or name.startswith('classifier')

        model = model.to(device)
        optimizer = opt_fn(filter(lambda p: p.requires_grad, model.parameters()))

        # Training loop
        for epoch in range(1, EPOCHS + 1):
            model.train()
            total_loss, correct, total = 0, 0, 0

            for x, y in train_loader:
                x, y = x.to(device), y.to(device)
                optimizer.zero_grad()
                out = model(x)
                loss = loss_fn(out, y)
                loss.backward()
                optimizer.step()

                total_loss += loss.item()
                correct += (out.argmax(1) == y).sum().item()
                total += y.size(0)

            print(f"  Epoch {epoch}/{EPOCHS} | TrainAcc {(correct / total):.4f}")

        # Save complete model checkpoint
        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss_config': lname,
            'opt_config': oname,
            'num_classes': num_classes,
            'epoch': EPOCHS
        }, os.path.join(config_dir, 'full_checkpoint.pth'))

        # Feature extraction
        encoder = make_encoder(model)
        X_train, y_train = extract_features(train_loader, encoder)
        X_val, y_val = extract_features(val_loader, encoder)

        # Save feature extractor
        torch.save(encoder.state_dict(), os.path.join(config_dir, 'feature_extractor.pth'))

        # Feature normalization
        norm = Normalizer('l2')
        X_train = norm.fit_transform(X_train)
        X_val = norm.transform(X_val)
        joblib.dump(norm, os.path.join(config_dir, 'feature_normalizer.joblib'))

        # Train and save classifiers
        svm = SVC(C=10, kernel='rbf', gamma='scale', probability=True)
        svm.fit(X_train, y_train)
        joblib.dump(svm, os.path.join(config_dir, 'svm_classifier.joblib'))

        lr = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)
        lr.fit(X_train, y_train)
        joblib.dump(lr, os.path.join(config_dir, 'logreg_classifier.joblib'))

        # Evaluation
        svm_results = evaluate(svm, f"{config_name} + rbfSVC", X_val, y_val)
        lr_results = evaluate(lr, f"{config_name} + Logistic", X_val, y_val)

        results.extend([svm_results, lr_results])
        print(f"    [SVM] Acc {svm_results['Accuracy']:.4f} | F1 {svm_results['F1']:.4f}")
        print(f"    [LR] Acc {lr_results['Accuracy']:.4f} | F1 {lr_results['F1']:.4f}")

        # Save config-specific results
        pd.DataFrame([svm_results, lr_results]).to_csv(
            os.path.join(config_dir, 'config_results.csv'), index=False)

# 5. Final Results and Experiment Summary

# Save all results
df_results = pd.DataFrame(results).sort_values('Accuracy', ascending=False)
df_results.to_csv(os.path.join(save_dir, 'all_results.csv'), index=False)

# Save experiment metadata
experiment_meta = {
    'timestamp': datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    'batch_size': BATCH_SIZE,
    'num_workers': NUM_WORKERS,
    'epochs': EPOCHS,
    'image_size': 224,
    'num_classes': num_classes,
    'loss_functions': [l[0] for l in loss_fns],
    'optimizers': [o[0] for o in opt_fns],
    'best_accuracy': df_results['Accuracy'].max()
}
joblib.dump(experiment_meta, os.path.join(save_dir, 'experiment_metadata.joblib'))

print("\n=== FINAL RESULTS ===")
print(df_results.to_string(index=False, float_format="%.4f"))
print(f"\n>>> All components saved to: {save_dir}")
print(f">>> Best accuracy: {df_results['Accuracy'].max():.4f}")


>>> All models will be saved to: /content/drive/MyDrive/AML-PROJECT/pretrained_cnn_Ly/EfficientNet_2_Noise
>>> Device: cuda
>>> Num classes: 200

=== CE_Adam ===
  Epoch 1/10 | TrainAcc 0.0112
  Epoch 2/10 | TrainAcc 0.2269
  Epoch 3/10 | TrainAcc 0.4537
  Epoch 4/10 | TrainAcc 0.6462
  Epoch 5/10 | TrainAcc 0.7900
  Epoch 6/10 | TrainAcc 0.8812
  Epoch 7/10 | TrainAcc 0.9369
  Epoch 8/10 | TrainAcc 0.9644
  Epoch 9/10 | TrainAcc 0.9850
  Epoch 10/10 | TrainAcc 0.9912


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


    [SVM] Acc 0.8000 | F1 0.7870
    [LR] Acc 0.7550 | F1 0.7491

=== CE_SGD ===
  Epoch 1/10 | TrainAcc 0.0106
  Epoch 2/10 | TrainAcc 0.1681
  Epoch 3/10 | TrainAcc 0.3694
  Epoch 4/10 | TrainAcc 0.5281
  Epoch 5/10 | TrainAcc 0.6906
  Epoch 6/10 | TrainAcc 0.8087
  Epoch 7/10 | TrainAcc 0.8988
  Epoch 8/10 | TrainAcc 0.9425
  Epoch 9/10 | TrainAcc 0.9694
  Epoch 10/10 | TrainAcc 0.9781


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


    [SVM] Acc 0.7800 | F1 0.7714
    [LR] Acc 0.6775 | F1 0.6669

=== LS_Adam ===
  Epoch 1/10 | TrainAcc 0.0163
  Epoch 2/10 | TrainAcc 0.2325
  Epoch 3/10 | TrainAcc 0.4863
  Epoch 4/10 | TrainAcc 0.6594
  Epoch 5/10 | TrainAcc 0.8056
  Epoch 6/10 | TrainAcc 0.9125
  Epoch 7/10 | TrainAcc 0.9444
  Epoch 8/10 | TrainAcc 0.9800
  Epoch 9/10 | TrainAcc 0.9869
  Epoch 10/10 | TrainAcc 0.9900


...

    [SVM] Acc 0.7475 | F1 0.7361
    [LR] Acc 0.6275 | F1 0.6169

=== FL_Adam ===
  Epoch 1/10 | TrainAcc 0.0187
  Epoch 2/10 | TrainAcc 0.2050
  Epoch 3/10 | TrainAcc 0.4575
  Epoch 4/10 | TrainAcc 0.6288
  Epoch 5/10 | TrainAcc 0.7825
  Epoch 6/10 | TrainAcc 0.9062
  Epoch 7/10 | TrainAcc 0.9481
  Epoch 8/10 | TrainAcc 0.9794
  Epoch 9/10 | TrainAcc 0.9856
  Epoch 10/10 | TrainAcc 0.9894


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


    [SVM] Acc 0.7800 | F1 0.7734
    [LR] Acc 0.7200 | F1 0.7109

=== FL_SGD ===
  Epoch 1/10 | TrainAcc 0.0112
  Epoch 2/10 | TrainAcc 0.1644
  Epoch 3/10 | TrainAcc 0.3769
  Epoch 4/10 | TrainAcc 0.5375
  Epoch 5/10 | TrainAcc 0.7100
  Epoch 6/10 | TrainAcc 0.8569
  Epoch 7/10 | TrainAcc 0.9275
  Epoch 8/10 | TrainAcc 0.9706
  Epoch 9/10 | TrainAcc 0.9794
  Epoch 10/10 | TrainAcc 0.9812


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


    [SVM] Acc 0.7600 | F1 0.7520
    [LR] Acc 0.6775 | F1 0.6696

=== FINAL RESULTS ===
           Variant  Accuracy     F1  Precision  Recall
  CE_Adam + rbfSVC    0.8000 0.7870     0.8221  0.8000
  LS_Adam + rbfSVC    0.7925 0.7753     0.8130  0.7925
  FL_Adam + rbfSVC    0.7800 0.7734     0.8380  0.7800
   CE_SGD + rbfSVC    0.7800 0.7714     0.8217  0.7800
   FL_SGD + rbfSVC    0.7600 0.7520     0.8073  0.7600
CE_Adam + Logistic    0.7550 0.7491     0.8193  0.7550
   LS_SGD + rbfSVC    0.7475 0.7361     0.7904  0.7475
FL_Adam + Logistic    0.7200 0.7109     0.7788  0.7200
LS_Adam + Logistic    0.7200 0.7138     0.7872  0.7200
 CE_SGD + Logistic    0.6775 0.6669     0.7466  0.6775
 FL_SGD + Logistic    0.6775 0.6696     0.7505  0.6775
 LS_SGD + Logistic    0.6275 0.6169     0.6940  0.6275

>>> All components saved to: /content/drive/MyDrive/AML-PROJECT/pretrained_cnn_Ly/EfficientNet_2_Noise
>>> Best accuracy: 0.8000
